In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup

from selenium.common.exceptions import NoSuchElementException

from PIL import Image
import io

import time

import pandas as pd
import utils.functions as F
import utils.data as data

In [2]:
house_id = 0

pref_sum_count = 0

In [3]:
for url in [data.urls['Yamagata']]:
    prefecture_name = "Yamagata"
    print(pref_sum_count, prefecture_name)
    pref_sum_count += 1

    url = url

    options = Options()
    options.add_argument('--headless')
    browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    browser.get(url)

    url = browser.current_url
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    a_elems = soup.find_all('div', attrs={'class' : 'property_unit-header'})

    data_list = []

    page_count = 0

    options = Options()
    options.add_argument('--headless')
    browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    browser.get(url)

    while True:
        url = browser.current_url
        print(url)
        #ブラウザの起動
        options = Options()
        options.add_argument('--headless')
        browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)

        browser.get(url)

        try:
            res = requests.get(url)
            soup = BeautifulSoup(res.content, 'html.parser')
            urls = F.get_urls(soup)
            house_info, house_id = F.get_index_info(urls, data_list, house_id)

            browser.find_element_by_link_text('次へ').click()
            if page_count % 10 == 0:
                print("pages:{0}".format(page_count))
                print('==============================================')
                time.sleep(60)
            time.sleep(60)

            page_count += 1

        except NoSuchElementException:
            browser.quit()
            break

    print('browser quit')

    houses_dict = {}
    img_list = []
    len(house_info)

    for house in house_info:
        # Pandasで加工しやすいようにKeyが一つの辞書に家の情報を変更する
        house_id, house_dict = F.edit_house_data(house)
        houses_dict[house_id] = house_dict
        # Houseの画像は変更が必要ないので、そのままリストに追加する
        # ただ、一つづつ取り出して追加する必要があるので、要注意
        img_list += house['imgs']

    attribute_df = pd.DataFrame(houses_dict).transpose()
    attribute_df.to_csv('csv/attribute_{filename}.csv'.format(filename = prefecture_name))
    imgs_df = pd.DataFrame(img_list)
    imgs_df.to_csv('csv/imgs_{filename}.csv'.format(filename = prefecture_name))



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


0 Hokkaido




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=010&bs=021&ta=01&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999
property's page URL :  /chukoikkodate/hokkaido_/sc_iwamizawa/nc_97138130/
success 0_0 https://suumo.jp/front/gazo/bukken/010/N010000/img/130/97138130/97138130_0001.jpg
success 0_1 https://suumo.jp/front/gazo/bukken/010/N010000/img/130/97138130/97138130_0002.jpg
success 0_2 https://suumo.jp/front/gazo/bukken/010/N010000/img/130/97138130/97138130_0003.jpg
success 0_3 https://suumo.jp/front/gazo/bukken/010/N010000/img/130/97138130/97138130_0005.jpg
success 0_4 https://suumo.jp/front/gazo/bukken/010/N010000/img/130/97138130/97138130_0014.jpg
success 0_5 https://suumo.jp/front/gazo/bukken/010/N010000/img/130/97138130/97138130_0023.jpg
success 0_6 https://suumo.jp/front/gazo/bukken/010/N010000/img/130/97138130/97138130_0007.jpg
success 0_7 https://suumo.jp/front/gazo/bukken/010/N010000/img/130/97138130/97138130_0004.



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


...

Driver has been saved in cache [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101]


property's page URL :  /chukoikkodate/hokkaido_/sc_sapporoshiminami/nc_96323615/
success 60_0 https://suumo.jp/front/gazo/bukken/010/N010000/img/615/96323615/96323615_0068.jpg
success 60_1 https://suumo.jp/front/gazo/bukken/010/N010000/img/615/96323615/96323615_0063.jpg
success 60_2 https://suumo.jp/front/gazo/bukken/010/N010000/img/615/96323615/96323615_0061.jpg
success 60_3 https://suumo.jp/front/gazo/bukken/010/N010000/img/615/96323615/96323615_0038.jpg
success 60_4 https://suumo.jp/front/gazo/bukken/010/N010000/img/615/96323615/96323615_0051.jpg
success 60_5 https://suumo.jp/front/gazo/bukken/010/N010000/img/615/96323615/96323615_0034.jpg
success 60_6 https://suumo.jp/front/gazo/bukken/010/N010000/img/615/96323615/96323615_0043.jpg
success 60_7 https://suumo.jp/front/gazo/bukken/010/N010000/img/615/96323615/96323615_0048.jpg
success 60_8 https://suumo.jp/front/gazo/bukken/010/N010000/img/615/96323615/96323615_0040.jpg
success 60_9 https://suumo.jp/front/gazo/bukken/010/N010000/img/



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=010&bs=021&ta=01&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999&pn=4
property's page URL :  /chukoikkodate/hokkaido_/sc_iwamizawa/nc_96346591/
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
property's page URL :  /chukoikkodate/hokkaido_/sc_usugun/nc_97058024/
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
property's page URL :  /chukoikkodate/hokkaido_/sc_sunagawa/nc_95721600/
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
property's page URL :  /chukoikkodate/hokkaido_/sc_tomakomai/nc_95765293/
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
property's page URL :  /chukoikkodate/hokkaido_/sc_hakodate/nc_967599

AttributeError: 'NoneType' object has no attribute 'attrs'